In [3]:
import os
os.chdir(r"C:\Users\lizzy\스파르타_파이썬\설무아튜터")
print(os.getcwd())  # 잘 바뀌었는지 확인


C:\Users\lizzy\스파르타_파이썬\설무아튜터


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from time import sleep
from urllib.parse import urljoin

BASE_URL = "https://www.aitimes.com"

def get_article_list(page: int = 1, section_code: str = "S1N1") -> list:
    """
    AI타임즈 뉴스 목록 페이지에서 기사 정보 추출
    """
    base_url = "https://www.aitimes.com/news/articleList.html"
    params = {
        "page": str(page),
        "sc_section_code": section_code,
        "view_type": "sm"
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36'
    }

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    articles = []

    # 기사 블록 선택
    for div in soup.select("div.altlist-webzine-content"):
        title_tag = div.select_one("h2.altlist-subject a")
        info_items = div.select("div.altlist-info-item")

        if not title_tag or len(info_items) < 2:
            continue

        title = title_tag.get_text(strip=True)
        relative_url = title_tag["href"]
        url = urljoin(BASE_URL, relative_url)   # ✅ 절대경로 변환
        reporter = info_items[0].get_text(strip=True)
        date = info_items[1].get_text(strip=True)

        articles.append({
            "title": title,
            "url": url,
            "reporter": reporter,
            "date": date
        })

    return articles


    
def get_article_content(url: str) -> str:
    """
    개별 기사 URL에서 본문 텍스트 추출
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        # ✅ id="article-view-content-div" 안의 모든 <p> 선택
        content_div = soup.find("article", id="article-view-content-div")
        if not content_div:
            return ""
        
        paragraphs = content_div.find_all("p")
        content = "\n".join(p.get_text(strip=True) for p in paragraphs)
        return content

    except Exception as e:
        print(f"본문 수집 실패 ({url}): {e}")
        return ""



def crawl_ai_news(pages: int = 3):
    """
    여러 페이지에 걸쳐 AI 뉴스 기사 수집
    """
    all_articles = []

    for page in range(1, pages + 1):
        print(f"📄 {page}페이지 수집 중...")
        articles = get_article_list(page)

        for article in articles:
            article["content"] = get_article_content(article["url"])
            all_articles.append(article)
        sleep(0.1)  # 페이지 단위로만 살짝 쉬기

    return all_articles


def save_to_json(data: list, filename: str = "aitimes_ai_news2.json"):
    """
    결과 데이터를 JSON 파일로 저장
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ {filename} 저장 완료 ({len(data)}개 기사)")


def summarize_articles(data: list):
    """
    날짜별 기사 수 등 간단한 통계 출력
    """
    df = pd.DataFrame(data)
    print("\n📊 날짜별 기사 수:")
    print(df["date"].value_counts())

    print("\n📰 최신 기사:")
    print(df.head(3)[["title", "date", "reporter"]])


if __name__ == "__main__":
    data = crawl_ai_news(pages=2)  # 원하는 페이지 수 조정
    save_to_json(data)
    summarize_articles(data)

📄 1페이지 수집 중...
📄 2페이지 수집 중...
✅ aitimes_ai_news2.json 저장 완료 (42개 기사)

📊 날짜별 기사 수:
date
               2
10-27 18:00    1
10-27 16:40    1
10-27 13:05    1
10-24 17:00    1
10-24 15:52    1
10-23 16:54    1
10-22 12:44    1
10-29 15:45    1
10-02 14:09    1
10-01 06:00    1
09-26 10:05    1
09-30 18:11    1
09-16 18:50    1
09-15 16:50    1
09-15 16:14    1
09-18 18:30    1
09-13 14:50    1
09-13 10:20    1
09-11 15:40    1
09-10 16:05    1
09-09 17:00    1
09-09 12:27    1
09-08 16:30    1
09-08 16:05    1
09-05 16:00    1
09-04 14:50    1
09-03 16:15    1
09-01 14:50    1
08-29 17:05    1
08-29 16:35    1
08-29 12:20    1
08-28 11:41    1
08-25 10:50    1
08-24 16:25    1
08-22 16:58    1
08-22 16:00    1
08-20 07:35    1
08-19 16:30    1
08-17 16:45    1
08-15 07:26    1
Name: count, dtype: int64

📰 최신 기사:
                                              title         date reporter
0                과기부, APEC 한-미 정상회담서 AI기술 협력 MOU 체결  10-29 15:45   장세민 기자
1  APEC CEO 서밋에 글로벌 기업인 1700명 참여